## Parameter setting and import libraries

In [1]:
from __future__ import print_function
#
import os
import glob
import re
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import threading
import time
from time import sleep
from tqdm import tqdm # if use notebook

import multiprocessing as mp
from multiprocessing import Event
import queue

from PIL import Image
import cv2
import imgaug as ia
from imgaug import augmenters as iaa
import random

parser = argparse.ArgumentParser()
parser.add_argument('--gpu_id', default=5)
parser.add_argument('--image_dir', default="/data/seanyu/cat_dog/dataset/")
parser.add_argument('--save_dir', default='./result')
parser.add_argument('--batch_size', default=4, type=int)
parser.add_argument('--do_augment', default=True, type = bool)
parser.add_argument('--epochs', default=100, type=int)
parser.add_argument('--lr', default=1e-4, type=float)
parser.add_argument('--image_size', default=(128,128,3), type = int)
parser.add_argument('--n_classes', default=2, type = int)
parser.add_argument('--n_batch', default=100, type = int)
parser.add_argument('--train_ratio', default=0.9, type = float)
parser.add_argument('--model_file_name', default = 'model.h5')
parser.add_argument('--n_threads', default = 4, type = int)
parser.add_argument('--dq_size', default = 6, type = int)

FLAGS = parser.parse_args([])
print(FLAGS)

/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  retu

Namespace(batch_size=4, do_augment=True, dq_size=6, epochs=100, gpu_id=5, image_dir='/data/seanyu/cat_dog/dataset/', image_size=(128, 128, 3), lr=0.0001, model_file_name='model.h5', n_batch=100, n_classes=2, n_threads=4, save_dir='./result', train_ratio=0.9)


/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
FLAGS.gpu_id = "4"
FLAGS.image_dir = "/data/seanyu/cat_dog/dataset/"

## Check path and load data

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = str(FLAGS.gpu_id)
import tensorflow as tf

if not os.path.exists(FLAGS.save_dir):
    os.makedirs(FLAGS.save_dir)

model_dir = FLAGS.save_dir + '/model'

"""  Get data """
d_train = FLAGS.image_dir + '/train/'
d_test = FLAGS.image_dir + '/test1/'

image_train_list = glob.glob(d_train + '*.jpg')
image_test_list = glob.glob(d_test + '*.jpg')

df_train = pd.DataFrame({'img_path': image_train_list})
df_test = pd.DataFrame({'img_path': image_test_list})

df_train['cate'] = df_train.img_path.apply(os.path.basename)
df_train['cate'] = [i.split(".")[0] for i in list(df_train.cate)]
df_train.cate = df_train.cate.replace({'dog': 0, 'cat': 1})

nb_epoch = FLAGS.epochs

df_train_0, df_val_0 = train_test_split(df_train[df_train['cate'] == 0], test_size = 1-FLAGS.train_ratio)
df_train_1, df_val_1 = train_test_split(df_train[df_train['cate'] == 1], test_size = 1-FLAGS.train_ratio)

df_val = pd.concat((df_val_0, df_val_1)).reset_index(drop = True)

del df_val_0, df_val_1

/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
try:
    import imgaug as ia
    from imgaug import augmenters as iaa
except:
    print("Import Error, Please make sure you have imgaug")
        
try:
    import sys
    sys.path.append("/mnt/deep-learning/usr/seanyu/common_tools/")
    from customized_imgaug_func import keypoint_func, img_channelswap
except:
    print("Warning, if you used customized imgaug function")
    
class Augmentation_Setup(object):  
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    lesstimes = lambda aug: iaa.Sometimes(0.2, aug)
    
    augmentation = iaa.Sequential([
        iaa.Fliplr(0.5, name="FlipLR"),
        iaa.Flipud(0.5, name="FlipUD"),
        iaa.OneOf([iaa.Affine(rotate = 90),
                   iaa.Affine(rotate = 180),
                   iaa.Affine(rotate = 270)]),
        sometimes(iaa.Affine(
                    scale = (0.8,1.2),
                    translate_percent = (-0.2, 0.2),
                    rotate = (-15, 15),
                    mode = 'wrap'
                    ))
    ])

## Data Generator

In [5]:
class GetDataset():
    def __init__(self, df_list, class_id, n_classes, f_input_preproc, image_size=(256,256,3), onehot=True, augmentation=None):
        
        self.df_list = df_list
        self.class_id = class_id
        self.n_classes = n_classes
        self.preproc = f_input_preproc
        self.image_size = image_size
        self.onehot = onehot
        self.aug = augmentation
        
        ## Init ##
        self.df_list = self.df_list.sample(frac=1.).reset_index(drop=True)
        self.current_index = 0
    
    def __len__(self):
        return len(self.df_list)
    
    def __getitem__(self, idx):
        
        img = self.load_image(img_path=self.df_list.iloc[self.current_index]['img_path'], image_size=self.image_size)
        
        if self.aug is not None:
            img = self.aug.augment_image(img)
            
        img = img.astype(np.float32)
        
        if self.preproc is not None:
            img = self.preproc(img)
        
        label = self.class_id
        if self.onehot:
             label = tf.keras.utils.to_categorical(label, num_classes=self.n_classes)
        
        self.current_index = (self.current_index + 1) % len(self.df_list)
        return img, label
    
    def __next__(self):
        return self.__getitem__(idx=self.current_index)
    
    @staticmethod
    def load_image(img_path, image_size):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (image_size[0], image_size[1]))
        return img
    
class Customized_dataloader():
    """
    1. Compose multiple generators together
    2. Make this composed generator into multi-processing function
    """
    def __init__(self, list_dataset, batch_size_per_dataset=16, queue_size=128, num_workers=0):
        """
        Args:
            - list_dataset: put generator object as list [gen1, gen2, ...]
            - batch_size_per_dataset: bz for each generator (total_batch_size/n_class)
            - queue_size: queue size
            - num_workers: start n workers to get data
        
        Action: Call with next
        """
        self.list_dataset = list_dataset
        self.batch_size_per_dataset = batch_size_per_dataset
        self.sample_queue = mp.Queue(maxsize = queue_size)
        
        self.jobs = num_workers
        self.events = list()
        self.workers = list()
        for i in range(num_workers):
            event = Event()
            work = mp.Process(target = enqueue, args = (self.sample_queue, event, self.compose_data))
            work.daemon = True
            work.start()
            self.events.append(event)
            self.workers.append(work)
        print("workers ready")
        
    def __next__(self):
        return self.sample_queue.get()
    
    def compose_data(self):
        while True:
            imgs, labels = [], []
            for z in range(self.batch_size_per_dataset):
                data = [next(i) for i in self.list_dataset]
                img, label = zip(*data)
                imgs.append(np.array(img))
                labels.append(np.array(label))
            yield np.concatenate(imgs), np.concatenate(labels)
    
    def stop_worker(self):
        for t in self.events:
            t.set()
        for i, t in enumerate(self.workers):
            t.join(timeout = 1)
        print("all_worker_stop")

# ----- #
def enqueue(queue, stop, gen_func):
    gen = gen_func()
    while True:
        if stop.is_set():
            return
        queue.put(next(gen))

In [6]:
def preproc(img):
    #return (img - img.min()) / (img.max() - img.min())
    return img / 255.

In [7]:
USE_RESNET_PREPROC = False
dog_train = GetDataset(df_list=df_train[df_train['cate'] == 0],
                       class_id=0, n_classes=2,
                       f_input_preproc=preproc if not USE_RESNET_PREPROC else tf.keras.applications.resnet50.preprocess_input,
                       augmentation=Augmentation_Setup.augmentation, 
                       onehot= True, 
                       image_size=(256,256,3))

cat_train = GetDataset(df_list=df_train[df_train['cate'] == 1], 
                       class_id=1, n_classes=2, 
                       f_input_preproc=preproc if not USE_RESNET_PREPROC else tf.keras.applications.resnet50.preprocess_input,
                       augmentation=Augmentation_Setup.augmentation, 
                       onehot= True, 
                       image_size=(256,256,3))

dog_valid = GetDataset(df_list=df_val[df_val['cate'] == 0], 
                       class_id=0, n_classes=2,
                       f_input_preproc=preproc if not USE_RESNET_PREPROC else tf.keras.applications.resnet50.preprocess_input,
                       augmentation=None, 
                       onehot= True, 
                       image_size=(256,256,3))

cat_valid = GetDataset(df_list=df_val[df_val['cate'] == 1], 
                       class_id=1, n_classes=2, 
                       f_input_preproc=preproc if not USE_RESNET_PREPROC else tf.keras.applications.resnet50.preprocess_input,
                       augmentation=None, 
                       onehot= True, 
                       image_size=(256,256,3))

In [8]:
valid_gen = Customized_dataloader([dog_valid, cat_valid], batch_size_per_dataset=FLAGS.batch_size//2, num_workers=2, queue_size=10)
x_val, y_val = [], []
for _ in tqdm(range(10)):
    a,b = next(valid_gen)
    x_val.append(a)
    y_val.append(b)
x_val = np.concatenate(x_val)
y_val = np.concatenate(y_val)
valid_gen.stop_worker()

print(x_val.shape)
print(y_val.shape)
print(y_val.sum(axis=0))

 10%|█         | 1/10 [00:00<00:01,  8.08it/s]

workers ready


100%|██████████| 10/10 [00:00<00:00, 23.67it/s]


all_worker_stop
(40, 256, 256, 3)
(40, 2)
[20. 20.]


## Build model

In [9]:
import sys
sys.path.append("/mnt/deep-learning/usr/seanyu/lab_mldl_tools/models/")

import keras
from keras_resnet_rebuilt.model import ResNet50, ResNet50V2

Using TensorFlow backend.


In [10]:
def build_model_graph(model_fn, norm_use, input_shape=(256,256,3), n_outputs=2):
    pretrain_modules = model_fn(include_top=False, input_shape=input_shape, norm_use=norm_use, weights='imagenet')
    gap = keras.layers.GlobalAveragePooling2D()(pretrain_modules.output)
    out = keras.layers.Dense(units=n_outputs, activation='softmax', name='output')(gap)
    model = keras.models.Model(inputs=[pretrain_modules.input], outputs=[out])
    return model

from keras.callbacks import Callback
class Logger(Callback):
    def __init__(self, n, gpu_id = 0):
        self.n = n   # print loss & acc every n epochs
        self.gpu_id = gpu_id

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.n == 0:
            # add what you need here
            train_loss = logs.get('loss')
            train_acc = logs.get('acc')
            valid_loss = logs.get('val_loss')
            valid_acc = logs.get('val_acc')
            print("GPU_ID: %s, epoch: %4d, loss: %0.5f, acc: %0.3f, val_loss: %0.5f, val_acc: %0.3f" \
                  % (self.gpu_id, epoch, 
                     train_loss, train_acc,
                     valid_loss, valid_acc))

import keras.backend as K
def train(device, model, generator):
    print("Start training on %s" % device)
    logger = Logger(n=1, gpu_id=device)
    with tf.Session(graph=graph) as session:
        tfk.set_session(session=session)
        model.fit_generator(generator, epochs=100, steps_per_epoch=100,
                  verbose=0, 
                  validation_data=(x_val, y_val), callbacks=[logger])


In [12]:
experiment_set = [ResNet50, ResNet50V2]
jobs = []
devices = ['/device:GPU:0', '/device:GPU:1']

for exp, device in zip(experiment_set, devices):
    with tf.Graph().as_default() as graph:
        with tf.device(device):
            model = build_model_graph(exp, "gn")
        optim = keras.optimizers.SGD(lr=1e-4, momentum=0.9, nesterov=True, decay=0.999)
        model.compile(loss='categorical_crossentropy', 
                      metrics=["accuracy"], 
                      optimizer=optim)
        jobs.append([graph, model])
print(jobs)

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder:0", shape=(7, 7, 3, 64), dtype=float32) is not an element of this graph.

In [12]:
import keras
keras.backend.clear_session()
pretrain_modules = ResNeXt50(include_top=False, input_shape=x_val[0].shape, norm_use="gn", weights='imagenet')
gap = keras.layers.GlobalAveragePooling2D()(pretrain_modules.output)
out = keras.layers.Dense(units=2, activation='softmax', name='output')(gap)
model = keras.models.Model(inputs=[pretrain_modules.input], outputs=[out])

optim = keras.optimizers.Adam(lr=1e-5, amsgrad=True)
model.compile(loss='categorical_crossentropy', 
              metrics=["accuracy"], 
              optimizer=optim)

In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9408        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_gn (GroupNorm)            (None, 128, 128, 64) 128         conv1_conv[0][0]                 
__________________________________________________________________________________________________
conv1_relu

## Start training

In [14]:
train_gen = Customized_dataloader([dog_train, cat_train], 
                                  batch_size_per_dataset=FLAGS.batch_size//2, 
                                  num_workers=4, queue_size=50)

workers ready


In [15]:
cb_list = [tf.keras.callbacks.ReduceLROnPlateau(factor=0.5,
                                                patience=4,
                                                min_lr=1e-12),
           tf.keras.callbacks.EarlyStopping(min_delta = 1e-4, 
                                            patience= 50)
          ]

model.fit_generator(train_gen,
                    epochs=FLAGS.epochs,
                    steps_per_epoch=FLAGS.n_batch, 
                    validation_data=(x_val, y_val),
                    #callbacks=cb_list
                    )

Epoch 1/100
100/100 [==============================] - 47s 473ms/step - loss: 0.6998 - acc: 0.5900 - val_loss: 0.7408 - val_acc: 0.5000
Epoch 2/100
100/100 [==============================] - 30s 299ms/step - loss: 0.6134 - acc: 0.6600 - val_loss: 0.6610 - val_acc: 0.6000
Epoch 3/100
100/100 [==============================] - 30s 299ms/step - loss: 0.6728 - acc: 0.5875 - val_loss: 0.6392 - val_acc: 0.6500
Epoch 4/100
100/100 [==============================] - 30s 301ms/step - loss: 0.6408 - acc: 0.6575 - val_loss: 0.6522 - val_acc: 0.5000
Epoch 5/100
100/100 [==============================] - 28s 284ms/step - loss: 0.6489 - acc: 0.6300 - val_loss: 0.5888 - val_acc: 0.6500
Epoch 6/100
100/100 [==============================] - 30s 300ms/step - loss: 0.6070 - acc: 0.6625 - val_loss: 0.6086 - val_acc: 0.7500
Epoch 7/100
100/100 [==============================] - 31s 309ms/step - loss: 0.5927 - acc: 0.6800 - val_loss: 0.6673 - val_acc: 0.7000
Epoch 8/100
100/100 [===========================

Epoch 61/100
100/100 [==============================] - 29s 290ms/step - loss: 0.1785 - acc: 0.9250 - val_loss: 0.8432 - val_acc: 0.8000
Epoch 62/100
100/100 [==============================] - 29s 290ms/step - loss: 0.1886 - acc: 0.9325 - val_loss: 0.3885 - val_acc: 0.8500
Epoch 63/100
100/100 [==============================] - 29s 287ms/step - loss: 0.1887 - acc: 0.9275 - val_loss: 0.4925 - val_acc: 0.8000
Epoch 64/100
100/100 [==============================] - 28s 284ms/step - loss: 0.2452 - acc: 0.8975 - val_loss: 1.0024 - val_acc: 0.8000
Epoch 65/100
100/100 [==============================] - 29s 292ms/step - loss: 0.1548 - acc: 0.9375 - val_loss: 0.7574 - val_acc: 0.8000
Epoch 66/100
100/100 [==============================] - 29s 291ms/step - loss: 0.1687 - acc: 0.9450 - val_loss: 0.5208 - val_acc: 0.8000
Epoch 67/100
100/100 [==============================] - 29s 290ms/step - loss: 0.1048 - acc: 0.9625 - val_loss: 0.6517 - val_acc: 0.8000
Epoch 68/100
100/100 [===================

In [16]:
train_gen.stop_worker()

all_worker_stop
